# Read and Write Mesh Data

In [1]:
from polymesh import PolyData
from polymesh.tri.trimesh import TriMesh
from polymesh.grid import Grid
from polymesh.space import StandardFrame
import numpy as np
import awkward as ak

A = StandardFrame(dim=3)
tri = TriMesh(size=(100, 100), shape=(10, 10), frame=A)
grid2d = Grid(size=(100, 100), shape=(10, 10), eshape='Q4', frame=A)
grid3d = Grid(size=(100, 100, 100), shape=(8, 6, 2), eshape='H8', frame=A)

mesh = PolyData(frame=A)
mesh['tri', 'T3'] = tri.move(np.array([0., 0., -50]))
mesh['grids', 'Q4'] = grid2d.move(np.array([0., 0., 150]))
mesh['grids', 'H8'] = grid3d

mesh['tri', 'T3'].pointdata['values'] = np.full(tri.coords().shape[0], 5.)
mesh['grids', 'Q4'].pointdata['values'] = np.full(
    grid2d.coords().shape[0], 10.)
mesh['grids', 'H8'].pointdata['values'] = np.full(
    grid3d.coords().shape[0], -5.)


In [2]:
mesh.topology()

TopologyArray
([[0, 1, 10], [10, 1, 11], [9, 19, ... 387], [384, 405, 408, 387, 385, 406, 409, 388]])

In [3]:
mesh.plot()

In [4]:
mesh['tri', 'T3'].pd.to_parquet("pdT3.parquet")
mesh['grids', 'Q4'].pd.to_parquet("pdQ4.parquet")
mesh['grids', 'H8'].pd.to_parquet("pdH8.parquet")
mesh['tri', 'T3'].cd.to_parquet("cdT3.parquet")
mesh['grids', 'Q4'].cd.to_parquet("cdQ4.parquet")
mesh['grids', 'H8'].cd.to_parquet("cdH8.parquet")


In [5]:
from polymesh.cells import T3, Q4, H8
from polymesh import PointData

frame=StandardFrame(dim=3)
mesh = PolyData(frame=frame)

"""pd = PointData(db=ak.from_parquet("pdT3.parquet"))
cd = T3(db=ak.from_parquet("cdT3.parquet"))
mesh['tri', 'T3'] = PolyData(pd, cd, frame=frame)"""

"""pd = PointData(db=ak.from_parquet("pdQ4.parquet"))
cd = Q4(db=ak.from_parquet("cdQ4.parquet"))
mesh['grids', 'Q4'] = PolyData(pd, cd, frame=frame)"""

pd = PointData(db=ak.from_parquet("pdH8.parquet"))
cd = H8(db=ak.from_parquet("cdH8.parquet"))
mesh['grids', 'H8'] = PolyData(pd, cd, frame=frame)


In [6]:
mesh.plot()

: 

: 

In [20]:
from neumann.array import minmax
minmax(mesh['grids', 'H8'].topology())

(-1862261504, 1952797216)

In [28]:
mesh.plot()

: 

: 

In [14]:
import pyvista as pv
from pyvista import themes

pv.set_plot_theme('paraview')

my_theme = themes.DarkTheme()
my_theme.color = 'red'
my_theme.lighting = False
my_theme.show_edges = True
my_theme.axes.box = True

pv.global_theme.load_theme(my_theme)


In [15]:
from pyvistaqt import BackgroundPlotter

plotter = BackgroundPlotter()
colors = ['red', 'green', 'blue']
for i, cb in enumerate(mesh.cellblocks()):
    data = cb.pointdata['values']
    plotter.add_mesh(cb.to_pv(), scalars=data)

: 

: 

In [1]:
plotter = BackgroundPlotter()
colors = ['red', 'green', 'blue']
for i, cb in enumerate(mesh.cellblocks()):
    m = cb.to_pv(scalars='values', fuse=False)[0]
    plotter.add_mesh(m)

NameError: name 'BackgroundPlotter' is not defined